# 计算机视觉算法综述

## YOLO1

YOLO1 是为了解决 RCNN，fast-RCNN 和 faster-RCNN 在目标检测项目落地中出现的一个大痛点 ： 速度问题！

其增强版本GPU中能跑45fps，简化版本155fps。

* YOLO主要特点是：
    * 速度快，能够达到实时的要求。在 Titan X 的 GPU 上 能够达到 45 帧每秒
    * 使用全图作为 Context 信息，背景错误（把背景错认为物体）比较少
    * 泛化能力强
    
mAP: mean Average Precision, 即各类别AP的平均值(平均精度的均值，各类别的AP的均值)  
AP: PR曲线下面积  
FPS : 指画面每秒传输帧数

![各类检测模型性能对比](./images/detection_contrast.jpg)

* **注意**YOLO其实是一种one-step的图片检测算法,而大家熟悉的Faster-RCNN其实是一种two-step算法
    * Faster-RCNN包含了 proposal 和 classifier 两个步骤

> YOLO将输入图像分成SxS个格子，若某个物体 Ground truth 的中心位置的坐标落入到某个格子，那么这个格子就负责检测出这个物体。  
![各类检测模型性能对比](./images/yolo1_3.png)


> YOLO 的核心思想就是利用整张图作为网络的输入，直接在输出层回归 bounding box（边界框） 的位置及bounding box（边界框） 所属的类别
![](./images/yolo1_1.jpg)

* 每个格子预测B个bounding box及其置信度(confidence score)，以及C个类别概率。
    * 每个 bounding box 除了要回归自身的位置(x, y, w, h)之外，还要附带预测一个 confidence 值。 一共**5**个值
    * 其次每个网格还要预测一个类别信息
    * 这个 confidence 代表了所预测的 box 中含有 object 的置信度和这个 box 预测的有多准这两重信息.
    * 定义为$Pr(Object) \times IOU^{truth}_{pred}, 其中Pr(Object)\in\{0,1\}$
        * 如果有 object 落在一个 方格(grid cell) 里，第一项取 1，否则取 0
        * 第二项是预测的 bounding box 和实际的 标记窗口(groundtruth) 之间的 IoU([检测评价函数](https://blog.csdn.net/eddy_zheng/article/details/52126641)) 值
            * IoU(检测评价函数): 简单来说就是 模型产生的目标窗口(bounding box) 和 原来标记窗口(groundtruth) 的交叠率,具体内容点上方链接
        * 用人话说:就是如果这个目标(object)恰好在这个方格里面,那么上述公式输出就是IOU的值(模型产生的目标窗口(bounding box) 和 原来标记窗口(groundtruth) 的交叠率);如果不在,对不起了,直接为0

    * 于是乎: 每个格子要预测B个bounding box及其置信度(confidence score)，以及C个类别概率。就要计算(5\*B+C)这些个值
    * 一共有SxS个格子,那么就要计算:  S x S x (5\*B+C) 个值
    
* 举例说明: 在 PASCAL VOC 中，图像输入为 448x448，取 S=7，B=2，一共有20 个类别（C=20），则输出就是 7x7x(2\*5 + 20) 的一个 tensor
    * 对于上面的S=7,就是将图像拆成7\*7个格子
    * B=2,就是说每个格子要预测2个bounding box
    * C=20,就是说每个格子要预测20个类别(**注意与每个 bounding box 的confidence做区别,confidence数量为1**)
    * 套公式: 7\*7\*(5\*2+20) = 1470 个 tensor

* 整个网络结构如下图
<!-- ![](./images/yolo1_4.jpg) -->

![](./images/yolo1_5.jpg)

* 当模型都训练好了以后,使用的时候,每个网格预测的 class 信息和 bounding box 预测的 confidence信息相乘，就得到每个 bounding box 的 class-specific confidence score:
    * ![](./images/yolo1_2.jpg)
    * 得到每个 box 的 class-specific confidence score 以后，设置阈值，滤掉得分低的 boxes，对保留的 boxes 进行 [NMS](https://blog.csdn.net/m0_37605642/article/details/98358864) 处理，就得到最终的检测结果。
    
    
* 重点内容
    * [边框回归(Bounding Box Regression)详解](https://blog.csdn.net/zijin0802034/article/details/77685438)
        * 这个博客中间有一些描述错误,不过不太碍事,学习过程中如果由更好的解释性文章,群里at我,做修正
        * [NMS——非极大值抑制](https://blog.csdn.net/shuzfan/article/details/52711706)
            * 这个讲的还可以
    * 每个 grid 有 30 维，这 30 维中，8 维是回归 box 的坐标，2 维是 box的 confidence，还有 20 维是类别。 
        * 在实现中，最主要的就是怎么设计损失函数，让这个三个方面得到很好的平衡。
        * 作者简单粗暴的全部采用了 sum-squared error loss 来做这件事。
            * 这种做法存在以下几个问题
                * 8维的 localization error 和20维的 classification error 同等重要显然是不合理的； **前8维应该更受重视**
                * 如果一个网格中没有 object（一幅图中这种网格很多），那么就会将这些网格中的 box 的 confidence push 到 0，相比于较少的有 object 的网格，这种做法是压倒性的导致网络不稳定甚至发散。
            * 解决办法
                * 更重视8维的坐标预测，给这些损失前面赋予更大的 loss weight, 例如在 pascal VOC 训练中取 5。
                * 对没有 object 的 box 的 confidence loss，赋予小的 loss weight，例如在 pascal VOC 训练中取 0.5。
                * 有 object 的 box 的 confidence loss 和类别的 loss 的 loss weight 正常取 1。
                * 对于相等的误差值，大物体误差对检测的影响应小于小物体误差对检测的影响,这是因为，相同的位置偏差占大物体的比例远小于同等偏差占小物体的比例,YOLO将物体大小的信息项（w和h）进行求平方根来改进这个问题，但并不能完全解决这个问题。 如下图：small bbox的横轴值较小，发生偏移时，反应到y轴上的loss（下图绿色）比big box(下图红色)要大。![](./images/yolo1_8.jpg)
                
    * 一个网格预测多个 box，希望的是每个 box predictor 专门负责预测某个 object。具体做法就是看当前预测的 box 与 ground truth box 中哪个 IoU 大，就负责哪个。这种做法称作 box predictor 的 specialization（专业化）。每个预测器会对特定（sizes,aspect ratio or classed of object）的ground true box预测的越来越好。（个人理解：IOU最大者偏移会更少一些，可以更快速的学习到正确位置）
    
* 损失函数
    * ![](./images/yolo1_6.jpg)
    
* 这个损失函数中： 
    * 只有当某个网格中有object的时候才对classification error进行惩罚。
    * 只有当某个box predictor对某个ground truth box负责的时候，才会对box的coordinate error进行惩罚，而对哪个ground truth box负责就看其预测值和ground truth box的IoU是不是在那个cell的所有box中最大。
    
* 损失函数设计 ![](./images/yolo1_7.jpg)
    
* YOLO与Fast R-CNN相比
    * YOLO与Fast R-CNN相比有较大的定位误差，与基于region proposal的方法相比具有较低的召回率。
    * 但是，YOLO在定位识别背景时准确率更高，而 Fast-R-CNN 的假阳性很高。
    * 基于此作者设计了 Fast-R-CNN + YOLO 检测识别模式，即先用R-CNN提取得到一组bounding box，然后用YOLO处理图像也得到一组bounding box。对比这两组bounding box是否基本一致，如果一致就用YOLO计算得到的概率对目标分类，最终的bouding box的区域选取二者的相交区域。
    * 这种组合方式将准确率提高了3个百分点。


* 大致流程
    * Resize成448*448，图片分割得到7*7网格(cell)
    * CNN提取特征和预测：卷积负责提特征。全链接部分负责预测：a) 7*7*2=98个bounding box(bbox) 的坐标x_{center},y_{center},w,h 和是否有物体的conﬁdence 。     b) 7*7=49个cell所属20个物体的概率。
    * 过滤bbox（通过nms）
    
* 训练过程
    * 预训练分类网络： 在 ImageNet 1000-class competition dataset上预训练一个分类网络，这个网络是Figure3中的前20个卷机网络+average-pooling layer+ fully connected layer （此时网络输入是224*224）。
    * 训练检测网络：转换模型去执行检测任务，《Object detection networks on convolutional feature maps》提到说在预训练网络中增加卷积和全链接层可以改善性能。在他们例子基础上添加4个卷积层和2个全链接层，随机初始化权重。检测要求细粒度的视觉信息，所以把网络输入也又224*224变成448*448。见Figure3。 
        * 幅图片分成7x7个网格(grid cell)，某个物体的中心落在这个网格中此网格就负责预测这个物体。![](./images/yolo1_3.png)
        * 最后一层输出为 （7*7）*30的维度。每个 1*1*30的维度对应原图7*7个cell中的一个，1*1*30中含有类别预测和bbox坐标预测。总得来讲就是让网格负责类别信息，bounding box主要负责坐标信息(部分负责类别信息：confidence也算类别信息)。具体如下：
            * 每个网格（1*1*30维度对应原图中的cell）要预测2个bounding box （图中黄色实线框）的坐标（x_{center},y_{center},w,h） ，其中：中心坐标的x_{center},y_{center} 相对于对应的网格归一化到0-1之间，w,h用图像的width和height归一化到0-1之间。 每个bounding box除了要回归自身的位置之外，还要附带预测一个confidence值。 这个confidence代表了所预测的box中含有object的置信度和这个box预测的有多准两重信息：confidence = Pr(Object) \ast IOU^{truth}_{pred}。其中如果有ground true box(人工标记的物体)落在一个grid cell里，第一项取1，否则取0。 第二项是预测的bounding box和实际的ground truth box之间的IOU值。即：每个bounding box要预测 x_{center},y_{center},w,h,confidence,共5个值 ，2个bounding box共10个值，对应 1*1*30维度特征中的前10个。![](./images/yolo1_9.jpg)  ![](./images/yolo1_10.jpg)
            * 每个网格还要预测类别信息，论文中有20类。7x7的网格，每个网格要预测2个 bounding box 和 20个类别概率，输出就是 7x7x(5x2 + 20)  。 (通用公式： SxS个网格，每个网格要预测B个bounding box还要预测C个categories，输出就是S x S x (5*B+C)的一个tensor。 注意：class信息是针对每个网格的，confidence信息是针对每个bounding box的） ![](./images/yolo1_11.jpg)

* 测试过程
    * Test的时候，每个网格预测的class信息( Pr(Class_i | Object)  )和bounding box预测的confidence信息( Pr(Object) \ast IOU^{truth}_{pred} ) 相乘，就得到每个bounding box的class-specific confidence score。
        * 等式左边第一项就是每个网格预测的类别信息，第二三项就是每个bounding box预测的confidence。这个乘积即encode了预测的box属于某一类的概率，也有该box准确度的信息。 ![](./images/yolo1_12.jpg)
        * 对每一个网格的每一个bbox执行同样操作： 7x7x2 = 98 bbox  （每个bbox既有对应的class信息又有坐标信息） ![](./images/yolo1_13.jpg)  ![](./images/yolo1_14.jpg)
    * 得到每个bbox的class-specific confidence score以后，设置阈值，滤掉得分低的boxes，对保留的boxes进行NMS处理，就得到最终的检测结果。 ![](./images/yolo1_15.jpg)


* YOLO的缺点
    * YOLO对相互靠的很近的物体，还有很小的群体 检测效果不好，这是因为一个网格中只预测了两个框，并且只属于一类。
    * 对测试图像中，同一类物体出现的新的不常见的长宽比和其他情况是。泛化能力偏弱。
    * 由于损失函数的问题，定位误差是影响检测效果的主要原因。尤其是大小物体的处理上，还有待加强。

## YOLO2

* 与VOLO_v1对比
    * 相比v1提高了训练图像的分辨率；
    * 引入了faster rcnn中anchor box的思想，对网络结构的设计进行了改进，
    * 输出层使用卷积层替代YOLO的全连接层
    * 联合使用coco物体检测标注数据和imagenet物体分类标注数据训练物体检测模型
    * **相比YOLO1，YOLO9000在识别种类、精度、速度、和定位准确性等方面都有大大提升。**
    * YOLO1与Fast R-CNN相比有较大的定位误差，与基于region proposal的方法相比具有较低的召回率。因此YOLO v2主要改进是提高召回率和定位能力。
    * YOLO1 对相互靠的很近的物体，还有很小的群体检测效果不好，这是因为一个网格中只预测了两个框，并且只属于一类
    * 同一类物体出现的新的不常见的长宽比和其他情况时，泛化能力偏弱
    * 由于损失函数的问题，定位误差是影响检测效果的主要原因。尤其是大小物体的处理上，还有待加强。
    

* YOLOv2
    * 代表着目前业界最先进物体检测的水平，它的速度要快过其他检测系统（FasterR-CNN，ResNet，SSD），使用者可以在它的速度与精确度之间进行权衡。
    * 使用YOLO9000这一网络结构可以实时地检测超过 9000 种物体分类，这归功于它使用了 WordTree，通过 WordTree 来混合检测数据集与识别数据集之中的数据。
    * [工程代码地址](http://pjreddie.com/darknet/yolo/)

* 联合训练算法的基本思路就是
    * 联合训练算法的基本思路就是：同时在检测数据集和分类数据集上训练物体检测器（Object Detectors ），用检测数据集的数据学习物体的准确位置，用分类数据集的数据来增加分类的类别量、提升鲁棒性。
    * YOLO9000 就是使用联合训练算法训练出来的，他拥有 9000 类的分类信息，这些分类信息学习自ImageNet分类数据集，而物体位置检测则学习自 COCO 检测数据集。
    * [代码和预训练模型地址](http://pjreddie.com/yolo9000/)
    
* 改进策略
    * Batch Normalization 批标准化
        * Batch Normalization可以提升模型收敛速度，而且可以起到一定正则化效果，降低模型的过拟合。在YOLOv2中，每个卷积层后面都添加了Batch Normalization层，并且不再使用droput。使用Batch Normalization后，YOLOv2的mAP提升了2.4%。
    * High Resolution Classifier 高分辨率的分类器
        * 目前大部分的检测模型都会在先在ImageNet分类数据集上预训练模型的主体部分（CNN特征提取器），所以YOLOv2增加了在ImageNet数据集上使用448\*448来finetune分类网络这一中间过程（10 epochs），这可以使得模型在检测数据集上finetune之前已经适用高分辨率输入。使用高分辨率分类器后，YOLOv2的mAP提升了约4%。
    * Convolutionlal With Anchor Boxes 结合使用Anchor Boxes来



* 分类网络
    * ![](./images/YOLO2分类网络.png)

* 检测网络
    * ![](./images/YOLO2检测网络.png)

* 网络结构
    ![](./images/yolo2_1.png)

## YOLO3

![](./images/YOLO3.png)

YOLOv3没有太多的创新，主要是借鉴一些好的方案融合到YOLO里面。不过效果还是不错的，在保持速度优势的前提下，提升了预测精度，尤其是加强了对小物体的识别能力。

本文主要讲v3的改进，由于是以v1和v2为基础。YOLO3主要的改进有：调整了网络结构；利用多尺度特征进行对象检测；对象分类用Logistic取代了softmax。

* 新的网络结构Darknet-53

在基本的图像特征提取方面，YOLO3采用了称之为Darknet-53的网络结构（含有53个卷积层），它借鉴了残差网络residual network的做法，在一些层之间设置了快捷链路（shortcut connections）。

![](./images/YOLO3_1.png)

上图的Darknet-53网络采用256*256*3作为输入，最左侧那一列的1、2、8等数字表示多少个重复的残差组件。每个残差组件有两个卷积层和一个快捷链路，示意图如下：

![](./images/YOLO3_4.png)

* 利用多尺度特征进行对象检测

![](./images/YOLO3_2.png)

YOLO2曾采用passthrough结构来检测细粒度特征，在YOLO3更进一步采用了3个不同尺度的特征图来进行对象检测。

结合上图看，卷积网络在79层后，经过下方几个黄色的卷积层得到一种尺度的检测结果。相比输入图像，这里用于检测的特征图有32倍的下采样。比如输入是416*416的话，这里的特征图就是13*13了。由于下采样倍数高，这里特征图的感受野比较大，因此适合检测图像中尺寸比较大的对象。

为了实现细粒度的检测，第79层的特征图又开始作上采样（从79层往右开始上采样卷积），然后与第61层特征图融合（Concatenation），这样得到第91层较细粒度的特征图，同样经过几个卷积层后得到相对输入图像16倍下采样的特征图。它具有中等尺度的感受野，适合检测中等尺度的对象。

最后，第91层特征图再次上采样，并与第36层特征图融合（Concatenation），最后得到相对输入图像8倍下采样的特征图。它的感受野最小，适合检测小尺寸的对象。

* 9种尺度的先验框

![](./images/YOLO3_3.png)


* 对象分类softmax改成logistic
预测对象类别时不使用softmax，改成使用logistic的输出进行预测。这样能够支持多标签对象（比如一个人有Woman 和 Person两个标签）。

* 输入映射到输出

![](./images/YOLO3_5.png)

不考虑神经网络结构细节的话，总的来说，对于一个输入图像，YOLO3将其映射到3个尺度的输出张量，代表图像各个位置存在各种对象的概率。

我们看一下YOLO3共进行了多少个预测。对于一个416*416的输入图像，在每个尺度的特征图的每个网格设置3个先验框，总共有 13*13*3 + 26*26*3 + 52*52*3 = 10647 个预测。每一个预测是一个(4+1+80)=85维向量，这个85维向量包含边框坐标（4个数值），边框置信度（1个数值），对象类别的概率（对于COCO数据集，有80种对象）。

对比一下，YOLO2采用13*13*5 = 845个预测，YOLO3的尝试预测边框数量增加了10多倍，而且是在不同分辨率上进行，所以mAP以及对小物体的检测效果有一定的提升。


## 三个YOLO模型对照

![](./images/yolo_总.png)

## Deep SORT 物体跟踪

SORT跟踪算法到底在干什么？（以单目标跟踪为例说明如下）

物体跟踪是把物体跟踪预测的BBox与物体检测的BBox关联，然后用对应的物体检测BBox代表成功跟踪的BBox结果

# 定义目标

# 数据获取

# 数据理解

# 特征工程

# 数据保存

# 模型选择

* 运行（pyecharts 下运行）

`python3 main.py -c car -i ../media/test20s.avi`

# 模型评估

# 模型优化

# 模型应用